<a href="https://colab.research.google.com/github/marco-siino/EXIST2024/blob/main/EXIST_2024_Task_1_Run1_2_Mistral7B_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In the Run 1 the starting and the ending tag "s" within the prompt is used for any few shot sample. In the Run 2 the "s" tag is used only once.

Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [1]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets
!pip install -U deep-translator

import datasets
from datasets import load_dataset
from deep_translator import GoogleTranslator
import json
import re
import random
import numpy as np
import tqdm.notebook as tqdm

# Seed to shuffle the json training set.
seed_value = 42
random.seed(seed_value)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 144.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 283.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 220.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 299.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.71-cp310-cp310-linux_x86_64.whl size=55352288 sha256=44db98e7cb9f73ce0787df2c837317274f478eb8d9af73d86c85863f2b0eae11
  Stored in directory: /tmp/pip-ephem-wheel-cache-p_qwmjo3/wheels/5d/35/d3/b4ab42b4528cc5e1f9b1469c4b4076de51e2f3ceb50506b424
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
 

Downloading an instruction-finetuned Mistral model.

In [ ]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4, #16, # CPU cores
    n_batch=800, #8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

# Download the dataset for the three subtasks.

In [ ]:
# Subtask 1.
!wget 'https://raw.githubusercontent.com/marco-siino/EXIST2024/main/dataset/EXIST2023_test_clean.json'
!wget 'https://raw.githubusercontent.com/marco-siino/EXIST2024/main/dataset/EXIST2024_training.json'
!wget 'https://raw.githubusercontent.com/marco-siino/EXIST2024/main/dataset/EXIST2024_training_task1_gold_hard.json'


train_path = 'EXIST2024_training.json'
train_gold_path = 'EXIST2024_training_task1_gold_hard.json'
test_path = 'EXIST2023_test_clean.json'

# Create few-shot samples from training set.

In [ ]:
 prompt_context = "[INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discrimination occurs (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behaviour)? Reply only with YES or NO.\n"


In [ ]:
def create_few_shot_samples_run1(json_set_path,nr_samples):

  nr_few_shot_samples = nr_samples

  few_shot_counter = 0

  few_shot_samples = ''

  with open(train_gold_path, 'r') as istr:
    train_gold_json = json.load(istr)
  indexed_data = {entry["id"]: entry for entry in train_gold_json}
  #print(indexed_data['100001'])
  with open(json_set_path, 'r') as istr:
    train_1_json = json.load(istr)
  #print(train_1_json[0])
  random.shuffle(train_1_json)
  for key, value in train_1_json.items():

    if few_shot_counter == nr_few_shot_samples:
      break
    try:
      label = indexed_data[key]['value']
    except:
      continue

    few_shot_counter+=1
    tweet = train_1_json[key]['tweet']
    print(train_1_json[key]['tweet'] + ' // ' + label)
    if train_1_json[key]['lang']=='es':
      tweet = GoogleTranslator(source='es', target='en').translate(train_1_json[key]['tweet'])

    few_shot_samples += '<s>'+prompt_context+"TWEET:"+tweet + ' [/INST] \n ' + label + "\n</s> \n\n "

    if few_shot_counter == nr_few_shot_samples:
        break

  return few_shot_samples

In [ ]:
def create_few_shot_samples_run2(json_set_path,nr_samples):

  nr_few_shot_samples = nr_samples

  few_shot_counter = 0

  few_shot_samples = ''

  with open(train_gold_path, 'r') as istr:
    train_gold_json = json.load(istr)
  indexed_data = {entry["id"]: entry for entry in train_gold_json}
  #print(indexed_data['100001'])
  with open(json_set_path, 'r') as istr:
    train_1_json = json.load(istr)
  #print(train_1_json[0])
  random.shuffle(train_1_json)
  for key, value in train_1_json.items():

    if few_shot_counter == nr_few_shot_samples:
      break
    try:
      label = indexed_data[key]['value']
    except:
      continue

    few_shot_counter+=1
    tweet = train_1_json[key]['tweet']
    print(train_1_json[key]['tweet'] + ' // ' + label)
    if train_1_json[key]['lang']=='es':
      tweet = GoogleTranslator(source='es', target='en').translate(train_1_json[key]['tweet'])

    few_shot_samples += prompt_context+"TWEET:"+tweet + ' [/INST] \n ' + label + "\n \n\n "

    if few_shot_counter == nr_few_shot_samples:
        break

  return few_shot_samples

In [ ]:
few_shot_samples_run1 = create_few_shot_samples_run1(train_path,10)

@TheChiflis Ignora al otro, es un capullo.El problema con este youtuber denuncia el acoso... cuando no afecta a la gente de izquierdas. Por ejemplo, en su video sobre el gamergate presenta como "normal" el acoso que reciben Fisher, Anita o Zöey cuando hubo hasta amenazas de bomba. // YES
@ultimonomada_ Si comicsgate se parece en algo a gamergate pues muy bien por el acoso. Y si se está haciendo un sabotaje porque hay personajes que no os gustan entonces gracias por darme la razón. Sois unos lloricas ofendidos. // NO
@Steven2897 Lee sobre Gamergate, y como eso ha cambiado la manera en la cual nos comunicamos en el internet. Los fanboys de Halo están tóxicos pero los fanboys de otras comunidades/juegos también han querido coger pauta con eso 🤷🏾‍♂️ // NO
@novadragon21 @icep4ck @TvDannyZ Entonces como así es el mercado lo mejor no es hacer algo para cambiarlo y seguir alimentando el machismo en los consumidores en lugar apoyar a gente como las víctimas del gamergate.Acerca de lo otro, el "

In [ ]:
few_shot_samples_run2 = create_few_shot_samples_run2(train_path,10)

In [ ]:
print(few_shot_samples_run1)

<s>[INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discrimination occurs (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behaviour)? Reply only with YES or NO.
TWEET:@TheChiflis Ignore the other one, he's an idiot. The problem with this YouTuber is that he denounces harassment... when it doesn't affect people on the left. For example, in his video about gamergate he presents as "normal" the harassment that Fisher, Anita or Zöey receive when there were even bomb threats. [/INST] 
 YES
</s> 

 <s>[INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discrimination occurs (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behaviour)? Reply only with YES or NO.
TWEET:@ultimonomada_ If comicsgate is anything like gamergate, then good for the harassment. And if sabotage is being done because there are characters that you don't like, then thank

In [ ]:
print(few_shot_samples_run2)

# Run!

In [ ]:
replies_list = ['YES','NO']
counter = 0

# List to store the results.
json_result_run1 = []


with open('answer.txt', 'w') as f:
        f.write('')

# simple JSON loading
with open(test_path, 'r') as istr:
    test_json = json.load(istr)
num_sample = len(test_json)
#print(num_sample)

#print(train_1_json[0])
for key, value in test_json.items():
  counter+=1
  tweet = test_json[key]['tweet']
  if test_json[key]['lang']=='es':
      tweet = GoogleTranslator(source='es', target='en').translate(test_json[key]['tweet'])

  current_sample = '[INST]\n'+tweet+'\n[/INST]'
  prompt = few_shot_samples_run1+current_sample

  #print(prompt)

  response = lcpp_llm(
        prompt=prompt,
        temperature= 0.2,
        logprobs=1,
        #max_tokens =1
      )

  #print(response)

  answer = str(response["choices"][0]["text"]).strip().upper()
  #print(answer)
  answer = answer[:3]
  #answer = answer.split()[0]
  # Sometime output contains a '.' remove it!
  #answer = answer.replace('.','')

  # If the predicted word is not in emotion list just replace with neutral.
  if answer not in replies_list:
        answer = 'NO'

  #current_sample += answer + " \n "

  print("GENERATED: "+ current_sample+'\n'+answer)

  current_element = {
        "id": test_json[key]['id'],
        "value": answer,
        "test_case": "EXIST2024"
    }
  json_result_run1.append(current_element)

with open("task1_hard_badrock_1.json", "w") as json_file:
    json.dump(json_result_run1, json_file, indent=4)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1255.65 ms
llama_print_timings:      sample time =      11.26 ms /    16 runs   (    0.70 ms per token,  1421.46 tokens per second)
llama_print_timings: prompt eval time =     953.35 ms /    31 tokens (   30.75 ms per token,    32.52 tokens per second)
llama_print_timings:        eval time =    1825.91 ms /    15 runs   (  121.73 ms per token,     8.22 tokens per second)
llama_print_timings:       total time =    3000.86 ms /    46 tokens


GENERATED: [INST]
@Eurogamer_es All gamergate from development to game forums, classic in the world of video games.
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1255.65 ms
llama_print_timings:      sample time =       8.84 ms /    16 runs   (    0.55 ms per token,  1809.95 tokens per second)
llama_print_timings: prompt eval time =     554.04 ms /    32 tokens (   17.31 ms per token,    57.76 tokens per second)
llama_print_timings:        eval time =    1479.82 ms /    15 runs   (   98.65 ms per token,    10.14 tokens per second)
llama_print_timings:       total time =    2169.46 ms /    47 tokens


GENERATED: [INST]
@ArCaNgEl__23 @Benzenazi Man, it's not comparable, look at Gamergate.
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1255.65 ms
llama_print_timings:      sample time =       9.09 ms /    16 runs   (    0.57 ms per token,  1760.18 tokens per second)
llama_print_timings: prompt eval time =     577.26 ms /    24 tokens (   24.05 ms per token,    41.58 tokens per second)
llama_print_timings:        eval time =    1508.51 ms /    15 runs   (  100.57 ms per token,     9.94 tokens per second)
llama_print_timings:       total time =    2204.03 ms /    39 tokens


...

GENERATED: [INST]
AUGUST 9 Find out everything! 🎙 You can watch the entire segment of 'El Bla, Bla, Bla' with HÉCTOR JOAQUÍN in the #Podcasts section of the LaMusica app. 📲#HectorJoaquin #LaManada #ElBlaBlaBla #LaMusica @la_musica @zeta93fm https://t.co/ZDZM4Qs1zx https://t.co/zqMjKMgN4r
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1255.65 ms
llama_print_timings:      sample time =       9.32 ms /    16 runs   (    0.58 ms per token,  1717.29 tokens per second)
llama_print_timings: prompt eval time =     291.88 ms /    89 tokens (    3.28 ms per token,   304.92 tokens per second)
llama_print_timings:        eval time =    1557.88 ms /    15 runs   (  103.86 ms per token,     9.63 tokens per second)
llama_print_timings:       total time =    2139.63 ms /   104 tokens


GENERATED: [INST]
@Fistroman1 @ElioGatsby Without proof?? What was it in public? What did we all see, what are you telling me? Don't mix churras with merinas. What from now until you come to me with the metoo and the notallmen there's little left and I don't feel like it end up really arguing, I see it coming.
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1255.65 ms
llama_print_timings:      sample time =      10.11 ms /    16 runs   (    0.63 ms per token,  1582.90 tokens per second)
llama_print_timings: prompt eval time =     281.86 ms /    83 tokens (    3.40 ms per token,   294.48 tokens per second)
llama_print_timings:        eval time =    1528.62 ms /    15 runs   (  101.91 ms per token,     9.81 tokens per second)
llama_print_timings:       total time =    2085.57 ms /    98 tokens


GENERATED: [INST]
@EstefaniaVeloz ...With the legal process, not to set a person on fire (I don't even know who they are in this case). There are women who are victims but there are also corrupt, violent and victimizers. The #MeToo movement has already fulfilled its mission. What's next? How to advance without it being a weapon of whim?
[/INST]
NO


In [ ]:
replies_list = ['YES','NO']
counter = 0

# List to store the results.
json_result_run2 = []

# simple JSON loading
with open(test_path, 'r') as istr:
    test_json = json.load(istr)
num_sample = len(test_json)
#print(num_sample)

#print(train_1_json[0])
for key, value in test_json.items():
  counter+=1
  tweet = test_json[key]['tweet']
  if test_json[key]['lang']=='es':
      tweet = GoogleTranslator(source='es', target='en').translate(test_json[key]['tweet'])

  current_sample = '[INST]\n'+tweet+'\n[/INST]'
  prompt = few_shot_samples_run2+current_sample

  #print(prompt)

  response = lcpp_llm(
        prompt=prompt,
        temperature= 0.2,
        logprobs=1,
        #max_tokens =1
      )

  #print(response)

  answer = str(response["choices"][0]["text"]).strip().upper()
  #print(answer)
  answer = answer[:3]
  #answer = answer.split()[0]
  # Sometime output contains a '.' remove it!
  #answer = answer.replace('.','')

  # If the predicted word is not in emotion list just replace with neutral.
  if answer not in replies_list:
        answer = 'NO'

  #current_sample += answer + " \n "

  print("GENERATED: "+ current_sample+'\n'+answer)

  current_element = {
        "id": test_json[key]['id'],
        "value": answer,
        "test_case": "EXIST2024"
    }
  json_result_run2.append(current_element)

with open("task1_hard_badrock_2.json", "w") as json_file:
    json.dump(json_result_run2, json_file, indent=4)


In [ ]:
!zip -r exist2024_badrock.zip task1_hard_badrock_1.json task1_hard_badrock_2.json

  adding: answer.txt (deflated 94%)


In [ ]:
from google.colab import files
files.download('exist2024_badrock.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>